## Неделя 2. Понедельник
### Обучение с учителем

### Применение базовых методов классификации

In [ ]:
import pandas as pd
import numpy as np
import joblib

#### 0. Ознакомьтесь с датасетом

In [36]:
df = pd.read_csv('heart.csv')
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40.0,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49.0,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37.0,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48.0,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54.0,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45.0,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68.0,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57.0,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57.0,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


* __Age__: age of the patient [years]
* __Sex__: sex of the patient [M: Male, F: Female]
* __ChestPainType__: chest pain type [TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic]
* __RestingBP__: resting blood pressure [mm Hg]
* __Cholesterol__: serum cholesterol [mm/dl]
* __FastingBS__: fasting blood sugar [1: if FastingBS > 120 mg/dl, 0: otherwise]
* __RestingECG__: resting electrocardiogram results [Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite * left ventricular hypertrophy by Estes' criteria]
* __MaxHR__: maximum heart rate achieved [Numeric value between 60 and 202]
* __ExerciseAngina__: exercise-induced angina [Y: Yes, N: No]
* __Oldpeak__: oldpeak = ST [Numeric value measured in depression]
* __ST_Slope__: the slope of the peak exercise ST segment [Up: upsloping, Flat: flat, Down: downsloping]
* __HeartDisease__: output class [1: heart disease, 0: Normal]

* Таргетом является столбец `HeartDisease`. Необходимо предсказать по имеющимся данным, есть ли проблемы с сердцем

#### 1. Небольшие рекомендации ниже 


* __Baseline pipeline (базовый пайплайн)__ - это простой пайплайн, который используется как отправная точка или точка сравнения при разработке и оценке более сложных моделей или алгоритмов. 

* Для этого сначала используйте самые простые идеи по заполнению пропусков(средними, медианами, модами) и кодированию категориальных данных, которые вам приходят в голову. 

* После того, как вы построите модели провалидируете их. Можно будет приступать к попыткам улучшить свою модель с помощью ваших идей - пробовать создавать новые фичи, кодировать данные по-другому, заполнять иначе NaN и тд

#### 2. Заполните пропущенные значения(`Imputing`), как считаете нужным.  

- Не забывайте памятку выше, сначала заполняйте самыми тривиальными идеями. Наприсер, средними, медианами и т.д

In [37]:
df.isnull().sum()

Age               10
Sex                0
ChestPainType      0
RestingBP          0
Cholesterol        0
FastingBS          0
RestingECG         0
MaxHR              0
ExerciseAngina     0
Oldpeak            0
ST_Slope           0
HeartDisease       0
dtype: int64

In [38]:
df['Age'] = df['Age'].fillna(df['Age'].mean())
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

##### 2.1 Оберните в `ColumnTransformer` свой `Imputing` данных. Проверьте корректность его работы. Для этого необходимо сделать:

1. Обучить и трансформировать свой `Imputer` с помощью `your_imputer.fit_transform` - на тренировочных данных
2. Заполнить с помощью `your_imputer.transform` - на тестовых данных

Убедитесь, что данные прошли через этап `Imputing'а` и пропусков в них больше нет

In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

ColumnTransformer` свой `Imputing` данных. Проверьте корректность его работы. Для этого необходимо сделать:

1. Обучить и трансформировать свой `Imputer` с помощью `your_imputer.fit_transform` - на тренировочных данных
2. Заполнить с помощью `your_imputer.transform` - на тестовых данных

Убедитесь, что данные прошли через этап `Imputing'а` и пропусков в них больше нет

In [40]:
X = df.drop('HeartDisease', axis=1) # без таргета
y = df['HeartDisease'] # таргет

# делим на с таргетом и без
X, y = df.drop('HeartDisease', axis=1), df['HeartDisease']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

num_features = ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']
cat_features = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

num_imputer = SimpleImputer(strategy='mean')           # заполнение средним для числовых
cat_imputer = SimpleImputer(strategy='most_frequent')  # заполнение модой для категориальных

my_imputer = ColumnTransformer(
    transformers = [('num', num_imputer, num_features), ('cat', cat_imputer, cat_features)])

X_train_imputed = my_imputer.fit_transform(X_train) # обучение и применение на train
X_valid_imputed = my_imputer.transform(X_valid) # только применение на valid

# X_train — обучающие признаки (80% от X)
# X_valid — валидационные признаки (20% от X)
# y_train — целевая переменная для обучения
# y_valid — целевая переменная для валидации
# X — признаки (features) все кроме 'HeartDisease'
# y — целевая переменная (target) df['HeartDisease']

# проверяем что пропусков нет
# преобразуем обратно в DataFrame, чтобы удобно проверить пропуски
X_train_imputed_df = pd.DataFrame(X_train_imputed, columns=num_features + cat_features)
X_valid_imputed_df = pd.DataFrame(X_valid_imputed, columns=num_features + cat_features)

print("Пропуски в X_train_imputed:\n", X_train_imputed_df.isnull().sum())
print("Пропуски в X_valid_imputed:\n", X_valid_imputed_df.isnull().sum())


# сохранить новывй датафрейм предобработанный
# преобразуем массивы обратно в DataFrame с именами колонок и индексами для сохранения порядка
X_train_imputed_df = pd.DataFrame(X_train_imputed, columns=num_features + cat_features, index=X_train.index)
X_valid_imputed_df = pd.DataFrame(X_valid_imputed, columns=num_features + cat_features, index=X_valid.index)

# добавим целевую переменную (таргет) обратно в датафреймы
X_train_imputed_df['HeartDisease'] = y_train
X_valid_imputed_df['HeartDisease'] = y_valid

# Сохраняем в csv без индекса, чтобы было чисто
# X_train_imputed_df.to_csv('aux/X_train_imputed.csv', index=False)
# X_valid_imputed_df.to_csv('aux/X_valid_imputed.csv', index=False)


Пропуски в X_train_imputed:
 Age               0
RestingBP         0
Cholesterol       0
FastingBS         0
MaxHR             0
Oldpeak           0
Sex               0
ChestPainType     0
RestingECG        0
ExerciseAngina    0
ST_Slope          0
dtype: int64
Пропуски в X_valid_imputed:
 Age               0
RestingBP         0
Cholesterol       0
FastingBS         0
MaxHR             0
Oldpeak           0
Sex               0
ChestPainType     0
RestingECG        0
ExerciseAngina    0
ST_Slope          0
dtype: int64


#### 3. Закодируйте категориальные переменные, как считаете нужным

* `OneHotEncoding` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)  
* `TargetEncoding` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.TargetEncoder.html)  
* `OrdinalEncoding` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html)  
* `CatBoostEncoding` (https://www.geeksforgeeks.org/categorical-encoding-with-catboost-encoder/)  

In [41]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler, OrdinalEncoder, TargetEncoder

ordinal_encoding_columns = ['Sex'] # Столбец, который планируем кодировать порядково, с помощью OrdinalEncoder 
one_hot_encoding_columns = ['ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'] # Столбец, который планируем кодировать с помощью OneHotEncoder 
standard_scaler_columns = ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak'] # Числовые столбцы, которые необходимо пронормировать


scaler_and_encoder = ColumnTransformer(
    [
        ('ordinal_encoding', OrdinalEncoder(), ordinal_encoding_columns),
        ('one_hot_encoding_columns', OneHotEncoder(sparse_output=False), one_hot_encoding_columns),
        ('scaling_num_columns', StandardScaler(), standard_scaler_columns)
    ],
    verbose_feature_names_out = False,
    remainder = 'passthrough' 
)

##### 3.1 Оберните в `ColumnTransformer` свой `Encoding` данных. Проверьте корректность его работы. 

In [42]:
# тут отдельно категориальные признаки кодируем

ordinal_encoding_columns = ['Sex']
one_hot_encoding_columns = ['ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

my_encoder = ColumnTransformer([
    ('ordinal_encoding', OrdinalEncoder(), ordinal_encoding_columns),
    ('one_hot_encoding', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), one_hot_encoding_columns)
], verbose_feature_names_out=False)

# кодируем тренировочные данные после импьютинга
X_train_encoded = my_encoder.fit_transform(X_train_imputed_df)

# кодируем валидационные данные
X_valid_encoded = my_encoder.transform(X_valid_imputed_df)

print("Форма X_train_encoded:", X_train_encoded.shape)
print("Форма X_valid_encoded:", X_valid_encoded.shape)


Форма X_train_encoded: (734, 13)
Форма X_valid_encoded: (184, 13)


#### 4. То же самое проделать с нормализацией данных

* `StandardScaler` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
* `MinMaxScaler` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)
* `RobustScaler` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)

#### 4.1 Оберните в `ColumnTransformer` свой `Scaling` данных, проверьте корректность работы.

In [43]:
# тут отдельно числовые признаки кодируем
standard_scaler_columns = ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']

my_scaler = ColumnTransformer([
    ('scaling_num_columns', StandardScaler(), standard_scaler_columns)
], verbose_feature_names_out=False)

# Применяем масштабирование к числовым признакам
X_train_scaled = my_scaler.fit_transform(X_train_imputed_df)
X_valid_scaled = my_scaler.transform(X_valid_imputed_df)

print("Форма X_train_scaled:", X_train_scaled.shape)
print("Форма X_valid_scaled:", X_valid_scaled.shape)


Форма X_train_scaled: (734, 6)
Форма X_valid_scaled: (184, 6)


#### 5. Соберите весь препроцессинг в общий Pipeline.

In [44]:
from sklearn.pipeline import Pipeline

# выделим типы признаков
num_features = ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']
cat_ordinal_features = ['Sex']
cat_onehot_features = ['ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

# отдельные пайплайны для каждого типа признаков

# числовой пайплайн: Импутинг + масштабирование
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# категориальный пайплайн: Импутинг + OrdinalEncoder
cat_ordinal_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])

# категориальный пайплайн: Импутинг + OneHotEncoder
cat_onehot_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# общий ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat_ord', cat_ordinal_pipeline, cat_ordinal_features),
    ('cat_onehot', cat_onehot_pipeline, cat_onehot_features)
], verbose_feature_names_out=False)

preprocessor = Pipeline([
    ('preprocessing', preprocessor)  # наш ColumnTransformer, собранный выше
])

##### 5.1 Прогоните свои данные через `preprocessor` и убедитесь, что ваши данные проходят через него корректно и уже готовы к ML-модели

In [45]:
# Обучаем preprocessor на тренировочных данных (fit + transform)
X_train_processed = preprocessor.fit_transform(X_train)

# Применяем обученный preprocessor к валидационным данным (только transform)
X_valid_processed = preprocessor.transform(X_valid)

# Проверяем форму результата (кол-во объектов и признаков)
print("Форма X_train_processed:", X_train_processed.shape)
print("Форма X_valid_processed:", X_valid_processed.shape)

# Проверяем, есть ли пропуски (NaN) после обработки

print("Пропуски в X_train_processed:", np.isnan(X_train_processed).sum())
print("Пропуски в X_valid_processed:", np.isnan(X_valid_processed).sum())


Форма X_train_processed: (734, 19)
Форма X_valid_processed: (184, 19)
Пропуски в X_train_processed: 0
Пропуски в X_valid_processed: 0


#### 6.ML-модели

* `LogisticRegression` (из `sklearn.linear_model`)  
* `LogisticRegression with regularization` (из `sklearn.linear_model`)  
* `KNeighborsClassifier` (из `sklearn.neighbors`)  
* `DecisionTree` (из `sklearn.tree`)  

##### 6.1 Обучите свой `Pipeline` с помощью метода `.fit()` с разными моделями.

In [46]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# preprocessor - готовый ColumnTransformer

models = {
    "LogisticRegression": LogisticRegression(random_state=42),
    "LogisticRegression_with_regularization": LogisticRegression(penalty='l2', C=1.0, solver='lbfgs', random_state=42),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTree": DecisionTreeClassifier(random_state=42)
}

for model_name, model in models.items():
    print(f"Обучение модели: {model_name}")
    ml_pipeline = Pipeline([
        ('preprocessor', preprocessor), 
        ('model', model)
    ])

    # обучаем модель
    ml_pipeline.fit(X_train, y_train)

    # оцениваем точность на валидационных данных
    score = ml_pipeline.score(X_valid, y_valid)
    print(f"Accuracy модели {model_name}: {score:.4f}\n")

Обучение модели: LogisticRegression
Accuracy модели LogisticRegression: 0.8859

Обучение модели: LogisticRegression_with_regularization
Accuracy модели LogisticRegression_with_regularization: 0.8859

Обучение модели: KNeighborsClassifier
Accuracy модели KNeighborsClassifier: 0.8804

Обучение модели: DecisionTree
Accuracy модели DecisionTree: 0.7717



#### 7. С помощью метода `.predict()` (на вход поступают только матрица признаков, без целевой переменной) предсказать значения на обучающей выборке (`X_train`) и валидационной выборке (`X_valid`).

In [47]:
# model_train_predict
# model_valid_predict

# предсказания на тренировочных данных
model_train_predict = ml_pipeline.predict(X_train)

# предсказания на валидационных данных
model_valid_predict = ml_pipeline.predict(X_valid)

print("Предсказания на тренировочных данных:", model_train_predict[:10])
print("Предсказания на валидационных данных:", model_valid_predict[:10])


Предсказания на тренировочных данных: [1 0 1 1 1 0 1 0 0 0]
Предсказания на валидационных данных: [1 1 1 0 1 1 0 0 0 1]


##### 7.1 С помощью функции оценки качества (`accuracy_score`) собрать следующую таблицу ниже

* значение функции на обучающих данных
* значение функции на валидационных данных 
    
Результатом выполнения этого пункта будет `DataFrame` формата: 
    
|  |train|valid|
|--|-----|-----|
|**LogReg**|  train_score  | valid_score    |
|**LogReg with l1**|  train_score  | valid_score    |
|**LogReg with l2**|  train_score  | valid_score    |
|**KNN**| train_score  |  valid_score   |
|**Tree**| train_score | valid_score    |

In [48]:
from sklearn.metrics import accuracy_score

# словарь моделей
models = {
    "LogReg": LogisticRegression(random_state=42),
    "LogReg with l1": LogisticRegression(penalty='l1', solver='liblinear', random_state=42),
    "LogReg with l2": LogisticRegression(penalty='l2', solver='lbfgs', random_state=42),
    "KNN": KNeighborsClassifier(),
    "Tree": DecisionTreeClassifier(random_state=42)
}

results = []

for model_name, model in models.items():
    # создаем pipeline с препроцессингом и моделью
    ml_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    # обучаем
    ml_pipeline.fit(X_train, y_train)
    
    # предсказания
    train_pred = ml_pipeline.predict(X_train)
    valid_pred = ml_pipeline.predict(X_valid)
    
    # считаем accuracy
    train_acc = accuracy_score(y_train, train_pred)
    valid_acc = accuracy_score(y_valid, valid_pred)
    
    results.append({
        'model': model_name,
        'train': train_acc,
        'valid': valid_acc
    })

results_df = pd.DataFrame(results).set_index('model')
results_df


,train,valid
model,,
LogReg,0.858311,0.885870
LogReg with l1,0.863760,0.885870
LogReg with l2,0.858311,0.885870
KNN,0.870572,0.880435
Tree,1.000000,0.771739


#### 8. Теперь реализуйте __кросс-валидацию__ с KFold=5 и выведите средний __score__

In [49]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import pandas as pd

models = {
    "LogReg": LogisticRegression(random_state=42),
    "LogReg with l1": LogisticRegression(penalty='l1', solver='liblinear', random_state=42),
    "LogReg with l2": LogisticRegression(penalty='l2', solver='lbfgs', random_state=42),
    "KNN": KNeighborsClassifier(),
    "SVC": SVC(random_state=42),
    "Tree": DecisionTreeClassifier(random_state=42)
}

kf = KFold(n_splits=5, shuffle=True, random_state=42)

results = {}

for name, model in models.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    scores = cross_val_score(pipeline, X, y, cv=kf, scoring='accuracy')
    results[name] = scores.mean()

df_results = pd.DataFrame.from_dict(results, orient='index', columns=['cross_val_score'])
df_results

# логистическая регрессия работает лучше для этого датасета

# train_test_split — разделение данных.
# SimpleImputer — заполнение пропусков.
# ColumnTransformer — применяет разные трансформации к разным колонкам.
# OrdinalEncoder, OneHotEncoder — кодирование категориальных данных.
# StandardScaler — масштабирование числовых признаков.
# Pipeline — объединение шагов препроцессинга и модели.
# LogisticRegression, KNeighborsClassifier, DecisionTreeClassifier, SVC — модели машинного обучения.
# cross_val_score и KFold — кросс-валидация.
# accuracy_score — метрика качества.

,cross_val_score
LogReg,0.856165
LogReg with l1,0.858345
LogReg with l2,0.856165
KNN,0.862693
SVC,0.863798
Tree,0.789736


|  |cross_val_score|
|--|-----|
|**LogReg**|  your_score |
|**LogReg with l1**|  your_score  |
|**LogReg with l2**|  your_score  |
|**KNN**| your_score  |
|**SVC**| your_score  |
|**Tree**| your_score |

<img src="https://icons.iconarchive.com/icons/icons8/windows-8/256/Programming-Github-icon.png" width=32 /> Пора сохранить изменения для __github__. 

1. Перейди в командной строке в папку, в которой расположен этот нотбук. 
2. Выполни команду `git add 06-01-task.ipynb`
3. Выполни команду `git commit -m "base models in progress"`
4. Выполни команду `git push`

##### 9. Теперь, когда вы проделали весь pipeline и обучили базовую модель, можно вернуться к началу и пробовать новые идеи и искать точки роста для ваших моделей, в том числе и добавление новых фичей

<img src="https://icons.iconarchive.com/icons/icons8/windows-8/256/Programming-Github-icon.png" width=32 /> Сохрани файл для __github__ и выполни команду `!git status` в ячейке ниже.


In [50]:
# code
# люди старше 60 сахар высокий True False
# 